# PROYECTO FINAL

### Importamos las librerias

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs 
from selenium import webdriver
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import mysql.connector as conn


In [2]:
from selenium.webdriver.chrome.options import Options
opciones=Options()
# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False   # si True, no aparece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('--incognito')              # incognito

### Scrapeamos la pagina web 'Monsieur Cuisine' , para sacar todas sus recetas con sus respectivos ingredientes

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()
URL='https://fr.monsieur-cuisine.com/es/filtros'
driver=webdriver.Chrome(PATH, options=opciones)
driver.get(URL)
time.sleep(5)
try:
    texto=driver.find_element(By.XPATH, '//*[@id="CybotCookiebotDialog"]/div/div[1]/button[1]') #aceptamos cookies
    texto.click()
    time.sleep(5)
except:
    pass



[WDM] - Downloading:  59%|█████████████████████████████████▌                       | 3.88M/6.58M [00:04<00:02, 985kB/s]

### Scrapeamos la pagina principal para poder sacar todos los títulos de recetas y poder sacar así los links de cada una de ellas

In [ ]:
nom_tit=[]
for e in driver.find_elements(By.XPATH, '//*[@id="app"]/div/main/div/div[2]/div/div/div[1]/div/div/div/div'):
    for x in e.find_elements(By.CLASS_NAME, 'v-card__title'):
        nom_tit.append(x.text)



In [ ]:
nom_tit

In [ ]:
link_recetas=[]
replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ñ", "n")
    )
for e in nom_tit:
    for a,b in replacements:
        e = e.lower().replace(' ', '-').replace('(', '').replace(')', '').replace(a,b)
    link_recetas.append(f'https://fr.monsieur-cuisine.com/es/recipe/{e}')

In [ ]:
len(link_recetas)

### Una vez tenemos los links de cada receta, generalizamos un proceso con Selenium para sacar la información que nos interesa para nuestra base de datos

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()
URL=link_recetas[10]
driver=webdriver.Chrome(PATH, options=opciones)
driver.get(URL)
time.sleep(5)
try:    
    texto=driver.find_element(By.XPATH, r'//*[@id="CybotCookiebotDialog"]/div/div[1]/button[1]') #aceptamos cookies
    texto.click()
    time.sleep(5)
except:
    pass



In [ ]:
titulo=driver.find_element(By.CLASS_NAME,'recipe-title').text        

In [ ]:
#dificultad= driver.find_element(By.XPATH,'//*[@id="app"]/div/main/div/div/div[1]/div[1]/div[2]/div/div[5]/span').text
dificultad=driver.find_element(By.XPATH,'//*[@id="app"]/div/main/div/div/div[1]/div[1]/div[2]/div/div[5]/span').text


In [ ]:
duración=driver.find_element(By.XPATH,'//*[@id="app"]/div/main/div/div/div[1]/div[1]/div[2]/div/div[4]/div[2]/span/span/span')


In [ ]:
ingredientes=driver.find_element(By.XPATH, '//*[@id="app"]/div/main/div/div/div[1]/div[2]/div[2]/div')


In [ ]:
driver.quit()

In [ ]:
titulo

In [ ]:
dificultad

In [ ]:
duración.text.strip()

In [ ]:
ingredientes.text.strip()

In [ ]:
for e in driver.find_elements(By.XPATH, '//*[@id="app"]/div/main/div/div/div[1]/div[1]/div[2]/div/div[3]/div/p/span'):

### Realizamos este proceso con cada una de las páginas

In [ ]:
def recetas(URL):
    #global titulos_lista
    #global ingredientes_lista
    #global dificultad_lista
    #global duracion_lista
    titulos_lista= []
    ingredientes_lista = []
    dificultad_lista = []
    duracion_lista = []
    #Titulo
    PATH=ChromeDriverManager().install()
    driver=webdriver.Chrome(PATH, options=opciones)
    driver.get(URL)
    time.sleep(5)
    try:    
        texto=driver.find_element(By.XPATH, '//*[@id="CybotCookiebotDialog"]/div/div[1]/button[1]') #aceptamos cookies
        texto.click()
        time.sleep(5)
    except:
        pass
    
    time.sleep(12)
    #titulo
    titulo=driver.find_element(By.CLASS_NAME,'recipe-title')
    titulos_lista.append(titulo.text.strip())
    #dificultad
    dificultad_lista.append(driver.find_element(By.XPATH,'//*[@id="app"]/div/main/div/div/div[1]/div[1]/div[2]/div/div[5]/span').text)
    #duracion
    duracion=driver.find_element(By.XPATH,'//*[@id="app"]/div/main/div/div/div[1]/div[1]/div[2]/div/div[4]/div[2]/span/span/span')
    duracion_lista.append(duracion.text.strip())
    #ingredientes
    ingredientes=driver.find_element(By.XPATH, '//*[@id="app"]/div/main/div/div/div[1]/div[2]/div[2]/div')
    ingredientes_lista.append(ingredientes.text.strip())
    
    driver.quit()
    
    return titulos_lista, duracion_lista, dificultad_lista, ingredientes_lista 


In [ ]:
link_recetas[1850:]

In [ ]:
r=Parallel(n_jobs = 6, verbose = True)(delayed(recetas)(url) for url in link_recetas[1850:])

In [ ]:
r

In [ ]:
carahuevo=carahuevo.append(r)

In [ ]:
carahuevo

### Creamos un archivo.csv para guardar todos los datos

In [ ]:
carahuevo.to_csv('recetas_terminado.csv', index=False)